In [1]:
from Bio import SeqIO,Phylo
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

#### Primers used

For CalCOFI - from Hong
MiSeq 16S F: GTGYCAGCMGCCGCGGTAA

~~MiSeq 16S R: CCGYCAATTCMTTTRAGT~~

For Training for 16S classifier, use 806 R GGACTACHVHHHTWTCTAAT


For Rachel's project, the following primers are used (same as CalCOFI??)
515 F GTGYCAGCMGCCGCGGTAA 926 R CCGYCAATTYMTTTRAGTTT

Illumina 18S V4
Illumina 18S V4 F: CCAGCASCYGCGGTAATTCC
Illumina 18S V4 R(pre 2015): ACTTTCGTTCTTGATYRA

Replace MiSeq 18S V4 R on September 2015 as below:
MiSeq 18S V4 (edited) R: ACTTTCGTTCTTGATYR

Illumina 18S V9
Illumina 18S V9 F: TTGTACACACCGCCC
Illumina 18S V9 R: CCTTCYGCAGGTTCACCTAC

For 16S V3V5 - from Beth
341F (5’-CCTACGGGNGGCWGCAG-3’) and 926R (5’-CCGTCAATTCMTTTRAGT-3’)

In [2]:
# Set work directory here
os.environ['workdir'] = '/usr/local/projdata/0568/projects/PLANKTON/illumina_aallen/db/Ocean_database'

### Train PR2 classifier

In [5]:
# Specify PR2 database files
PR_DB_ori = pd.read_csv(os.environ['workdir']+'/fasta_database_raw/PR2/pr2_version_4.11.1_merged.tsv',sep='\t')

In [6]:
PR_DB = PR_DB_ori[['genbank_accession','sequence','kingdom', 'supergroup', 'division', 'class', 'order', 'family', 'genus','species']]
PR_DB = PR_DB.loc[PR_DB['sequence'].dropna().index]
PR_DB = PR_DB.loc[PR_DB.genbank_accession.drop_duplicates().index].set_index('genbank_accession')

In [7]:
# Separate sequence from the tsv file and write into a fasta format
with open(os.environ['workdir']+'/fasta_database_raw/PR2/PR2_for_qiime2.fasta', "w") as handle:
    for ID in tqdm(PR_DB.index):
        handle.write(">"+ str(ID)+'\n'+PR_DB.sequence.loc[ID]+'\n')

100%|██████████| 164144/164144 [00:07<00:00, 20821.64it/s]


In [8]:
# Separate taxonomy information from the tsv file and write into a txt file
with open(os.environ['workdir']+'/fasta_database_raw/PR2/PR2_taxonomy.txt', "w") as handle:
    for ID in tqdm(PR_DB.index):
        handle.write(str(ID)+'\t kingdom_'+PR_DB['kingdom'].loc[ID]
                     +';supergroup_'+PR_DB['supergroup'].loc[ID]
                     +';division_'+PR_DB['division'].loc[ID]
                     +';class_'+PR_DB['class'].loc[ID]
                     +';order_'+PR_DB['order'].loc[ID]
                     +';family_'+PR_DB['family'].loc[ID]
                     +';genus_'+PR_DB['genus'].loc[ID]
                     +';species_'+PR_DB['species'].loc[ID]+'\n')

100%|██████████| 164144/164144 [00:52<00:00, 3113.41it/s]


In [38]:
# Convert these files to qiime2 compatible format
!qiime tools import \
  --type 'FeatureData[Sequence]' \
  --input-path $workdir/fasta_database_raw/PR2/PR2_for_qiime2.fasta \
  --output-path $workdir/qiime2_compatible_database/PR2/PR2_for_qiime2.qza

!qiime tools import \
  --type 'FeatureData[Taxonomy]' \
  --source-format HeaderlessTSVTaxonomyFormat \
  --input-path $workdir/fasta_database_raw/PR2/PR2_taxonomy.txt \
  --output-path $workdir/qiime2_compatible_database/PR2/PR2_taxonomy.qza

In [41]:
# Modify the sequence as needed. For V9 region only
!qiime feature-classifier extract-reads \
  --i-sequences $workdir/qiime2_compatible_database/PR2/PR2_for_qiime2.qza \
  --p-f-primer TTGTACACACCGCCC \
  --p-r-primer CCTTCYGCAGGTTCACCTAC \
  --o-reads $workdir/qiime2_compatible_database/PR2/PR2_for_qiime2_V9.qza

Saved FeatureData[Sequence] to: /usr/local/projdata/0568/projects/PLANKTON/illumina_aallen/db/Ocean_database//qiime2_compatible_database/PR2/PR2_for_qiime2_TAReuk.qza


In [ ]:
# Classifier for V9 region only
!qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads $workdir/qiime2_compatible_database/PR2/PR2_for_qiime2_V9.qza \
  --i-reference-taxonomy $workdir/qiime2_compatible_database/PR2/PR2_taxonomy.qza \
  --o-classifier $workdir/classifier/PR2_V9_classifier.qza

In [ ]:
# Modify the sequence as needed. For V4 region only
!qiime feature-classifier extract-reads \
  --i-sequences $workdir/qiime2_compatible_database/PR2/PR2_for_qiime2.qza \
  --p-f-primer CCAGCASCYGCGGTAATTCC \
  --p-r-primer ACTTTCGTTCTTGATYR \
  --o-reads $workdir/qiime2_compatible_database/PR2/PR2_for_qiime2_V4.qza

In [ ]:
# Classifier for V4 region only
!qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads $workdir/qiime2_compatible_database/PR2/PR2_for_qiime2_V4.qza \
  --i-reference-taxonomy $workdir/qiime2_compatible_database/PR2/PR2_taxonomy.qza \
  --o-classifier $workdir/classifier/PR2_V4_classifier.qza

In [ ]:
# Classifier for all regions
!qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads $workdir/qiime2_compatible_database/PR2/PR2_for_qiime2.qza \
  --i-reference-taxonomy $workdir/qiime2_compatible_database/PR2/PR2_taxonomy.qza \
  --o-classifier  $workdir/classifier/PR2_classifier_general.qza

### Train Silver 18S classifier

In [13]:
!qiime tools import \
  --type 'FeatureData[Sequence]' \
  --input-path $workdir/fasta_database_raw/Silva/silva132_99.fna \
  --output-path $workdir/qiime2_compatible_database/SILVA132/silva132_99_qiime2.qza

!qiime tools import \
  --type 'FeatureData[Taxonomy]' \
  --source-format HeaderlessTSVTaxonomyFormat \
  --input-path $workdir/fasta_database_raw/Silva/taxonomy_7_levels.txt \
  --output-path $workdir/qiime2_compatible_database/SILVA132/silva132_99_taxonomy.qza

In [ ]:
# Classifier for all regions
!qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads $workdir/qiime2_compatible_database/SILVA132/silva132_99_qiime2.qza \
  --i-reference-taxonomy $workdir/qiime2_compatible_database/SILVA132/silva132_99_taxonomy.qza \
  --o-classifier $workdir/classifier/SILVA132_classifier.qza

In [ ]:
# Train Silva 18S classifier specific for V4 region
!qiime feature-classifier extract-reads \
  --i-sequences $workdir/qiime2_compatible_database/SILVA132/silva132_99_qiime2.qza \
  --p-f-primer CCAGCASCYGCGGTAATTCC \
  --p-r-primer ACTTTCGTTCTTGATYR \
  --o-reads $workdir/qiime2_compatible_database/SILVA132/silva132_99_qiime2_TAReuk.qza

In [ ]:
!qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads $workdir/qiime2_compatible_database/SILVA132/silva132_99_qiime2_TAReuk.qza \
  --i-reference-taxonomy $workdir/qiime2_compatible_database/SILVA132/silva132_99_taxonomy.qza \
  --o-classifier $workdir/classifier/SILVA132_TAReuk_V4_classifier.qza

In [ ]:
# Train Silva 18S classifier specific for V9 region
!qiime feature-classifier extract-reads \
  --i-sequences $workdir/qiime2_compatible_database/SILVA132/silva132_99_qiime2.qza \
  --p-f-primer TTGTACACACCGCCC \
  --p-r-primer CCTTCYGCAGGTTCACCTAC \
  --o-reads $workdir/qiime2_compatible_database/SILVA132/silva132_99_qiime2_1389F_1510R.qza

In [ ]:
!qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads $workdir/qiime2_compatible_database/SILVA132/silva132_99_qiime2_1389F_1510R.qza \
  --i-reference-taxonomy $workdir/qiime2_compatible_database/SILVA132/silva132_99_taxonomy.qza \
  --o-classifier $workdir/classifier/silva132_99_V9_qiime2_1389F_1510R_classifier.qza

### Train PhytoRef 18S classifier

In [38]:
dict_phyto = SeqIO.to_dict(rec.upper() for rec in SeqIO.parse(os.environ['workdir']+'/fasta_database_raw/PhytoRef/PhytoRef_with_taxonomy.fasta','fasta'))

df_phylo = pd.DataFrame(index = ['Kingdom'
    ,'Supergroup'
    ,'Phylum'
    ,'Class'
    ,'Subclass'
    ,'Order'
    ,'Suborder'
    ,'Family'
    ,'Genus'
    ,'Species'])

for entry in dict_phyto.keys():
    temp = [tax for tax in entry.split('|')]
    df_phylo[temp[0]] = temp[1:]
    
df_phylo=df_phylo.T

for col in df_phylo.columns:
    df_phylo[col] = df_phylo[col].map(lambda x:col+'.'+x)

df_phylo['lineage'] = df_phylo.apply(lambda x: ';'.join(x.values.tolist()), axis=1) 

df_phylo.index = [x.replace('#','.') for x in df_phylo.index]

df_phylo[['lineage']].to_csv(os.environ['workdir']+'/fasta_database_raw/PhytoRef/Phyto_18S_taxonomy.txt',sep = '\t',header=False)

In [86]:
dict_phyto_clean = dict(zip(list(df_phylo.index),[dict_phyto[seq].seq for seq in dict_phyto.keys()]))
with open(os.environ['workdir']+'/fasta_database_raw/PhytoRef/PhytoRef_No_tax.fasta','w') as handle:
    for ID in tqdm(dict_phyto_clean):
        handle.write('>'+ID.replace('#','.')+'\n'+str(dict_phyto_clean[ID]).replace('X','-')+'\n')

100%|██████████| 6490/6490 [00:00<00:00, 126260.04it/s]


In [87]:
!qiime tools import \
  --type 'FeatureData[Sequence]' \
  --input-path $workdir/fasta_database_raw/PhytoRef/PhytoRef_No_tax.fasta \
  --output-path $workdir/qiime2_compatible_database/PhytoRef/PhytoRef_18S_qiime2.qza

!qiime tools import \
  --type 'FeatureData[Taxonomy]' \
  --source-format HeaderlessTSVTaxonomyFormat \
  --input-path $workdir/fasta_database_raw/PhytoRef/Phyto_18S_taxonomy.txt \
  --output-path $workdir/qiime2_compatible_database/PhytoRef/Phyto_18S_taxonomy.qza

In [88]:
!qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads $workdir/qiime2_compatible_database/PhytoRef/PhytoRef_18S_qiime2.qza \
  --i-reference-taxonomy $workdir/qiime2_compatible_database/PhytoRef/Phyto_18S_taxonomy.qza \
  --o-classifier $workdir/classifier/Phyto_Ref_classifier.qza

Saved TaxonomicClassifier to: classifier/Phyto_Ref_classifier.qza
